# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [9]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10ee2ca10>: 51, mem=True, ro=False: entities=9, map=2


In [10]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [11]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [12]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def cutoffs(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        
        clock = valuable.getObj(_tr.RootClock, 'Main')
        
        def cuts(ts):
            def fn(node):
                obj = node.key.object()
                m = node.key.shortName()
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return nodes

        ret = {}
        for n in cuts(ts1):
            ret[n.key._key] = n
        for n in cuts(ts2):
            ret[n.key._key] = n
        return ret.values()
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        nodes = self.cutoffs()
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.object().meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n] = tsAmend
            name = n.object().meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n] = ts2
            name = n.object().meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            txt = '\n'.join( [ '1. %s' % f for f in footnotes])
            displayMarkdown(txt)
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [13]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [14]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times: /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|42,279,742|25|42,279,742|GetValue
|Report:data|1|42,279,716|806|42,279,716|GetValue/Calc
|TradingContainer:NPV|11|42,095,721|303|3,826,883|GetValue
|TradingContainer:NPV|11|42,095,417|179|3,826,856|GetValue/Calc
|Portfolio:items|121|41,674,997|2,671|344,421|GetValue
|Portfolio:items|121|41,674,775|1,929|344,419|GetValue/Calc
|Workbook:items|1,104|39,357,133|25,921|35,649|GetValue
|Workbook:items|1,100|39,331,211|19,546|35,755|GetValue/Calc
|Report:cutoffs|1|22,082,609|39|22,082,609|GetValue
|Report:cutoffs|1|22,082,569|231|22,082,569|GetValue/Calc
|Root:Clocks|548|22,082,325|4,822,060|40,296|Context
|TradeOpenEvent:ticket|22,286|9,328,718|274,531|418|GetValue
|TradingTicket|1,014|9,054,186|9,054,186|8,929|Db.Get
|TradeOpenEvent|1,014|6,480,418|6,480,418|6,390|Db.Get
|Root:Start breaks|255|3,009,663|2,836,187|11,802|Context
|Root:End|278|2,784,824|2,646,094|10,017|Context
|Root:Start|270|2,315,956|2,134,096|8,577|Context
|Root:Amend Portfolio|255|2,247,560|2,052,510|8,813|Context
|Portfolio:children|242|2,222,458|4,118|9,183|GetValue
|Portfolio:children|121|2,218,340|2,207|18,333|GetValue/Calc
|Root:Activity Portfolio|263|2,020,722|1,855,903|7,683|Context
|Root:Amend MarketData|255|2,006,752|1,845,448|7,869|Context
|Root:Amend Trading|255|1,986,149|1,826,340|7,788|Context
|Root:Activity MarketData|255|1,968,687|1,809,652|7,720|Context
|Root:Activity Trading|263|1,850,812|1,702,378|7,037|Context
|PortfolioUpdateEvent:children|121|940,343|6,092|7,771|GetValue
|TradingBook|102|892,916|892,916|8,754|Db.Get
|Equity:NPV|15|415,194|370|27,679|GetValue
|Equity:NPV|15|414,823|231|27,654|GetValue/Calc
|MarketInterface:spot|15|388,960|329|25,930|GetValue
|MarketInterface:spot|15|388,630|228|25,908|GetValue/Calc
|ExternalRefData:state|15|362,682|361|24,178|GetValue
|ExternalRefData:state|15|362,321|240|24,154|GetValue/Calc
|RefData:state|15|361,135|312|24,075|GetValue
|RefData:state|15|360,823|256|24,054|GetValue/Calc
|_WorkItemEvent:book2|22,286|254,153|244,675|11|GetValue
|_WorkItemEvent:book1|22,286|251,940|251,940|11|GetValue
|Clock:cutoffs|2,480|234,429|19,643|94|GetValue
|TradeOpenEvent:quantity|22,286|216,655|216,655|9|GetValue
|Clock:cutoffs|20|214,957|337|10,747|GetValue/Calc
|Clock:parent|20|212,737|436|10,636|GetValue
|Clock:parent|20|212,301|344|10,615|GetValue/Calc
|_WorkItemEvent:item|11,143|181,590|172,489|16|GetValue
|TradeOpenEvent:action|11,143|165,801|165,801|14|GetValue
|TradeOpenEvent:premium|11,143|163,984|159,101|14|GetValue
|TradeOpenEvent:unitPrice|11,143|137,709|137,709|12|GetValue
|PortfolioUpdateEvent|12|124,960|124,960|10,413|Db.Get
|TradingBook:clock|2,200|124,289|47,461|56|GetValue
|Event:amends|11,340|122,378|122,378|10|GetValue
|RefDataUpdateEvent|9|104,832|104,832|11,648|Db.Get
|TradingBook:clock|1,100|76,828|26,671|69|GetValue/Calc
|TradingPortfolio|10|50,812|50,812|5,081|Db.Get
|Clock|5|35,202|35,202|7,040|Db.Get
|Equity:refdata|15|24,339|353|1,622|GetValue
|Equity:refdata|15|23,985|236|1,599|GetValue/Calc
|MarketInterface|2|22,414|22,414|11,207|Db.Get
|MarketInterface:source|15|22,147|285|1,476|GetValue
|MarketInterface:source|15|21,862|573|1,457|GetValue/Calc
|MarketDataSource|2|19,524|19,524|9,762|Db.Get
|ClockEvent|2|18,318|18,318|9,159|Db.Get
|Portfolio:books|231|18,194|3,791|78|GetValue
|Portfolio:clock|242|18,187|5,094|75|GetValue
|Portfolio:books|121|15,170|1,943|125|GetValue/Calc
|ClockEvent:parent|8|14,719|170|1,839|GetValue
|Portfolio:clock|121|13,093|2,786|108|GetValue/Calc
|MarketDataSource:clock|30|12,287|652|409|GetValue
|MarketDataSource:clock|15|11,635|334|775|GetValue/Calc
|Equity|2|9,100|9,100|4,550|Db.Get
|RootClock|1|5,156|5,156|5,156|Db.Get
|ForwardCashflow|1|4,882|4,882|4,882|Db.Get
|Entity:clock|40|1,929|786|48|GetValue
|Entity:clock|20|1,143|473|57|GetValue/Calc
|ForwardCashflow:NPV|11|859|293|78|GetValue
|MarketInterface:sourceName|15|713|311|47|GetValue
|RefDataUpdateEvent:data|58|588|588|10|GetValue
|ForwardCashflow:NPV|11|566|185|51|GetValue/Calc
|MarketInterface:sourceName|15|402|233|26|GetValue/Calc
|RootClock:cutoffs|52|396|396|7|GetValue
|Equity:assetName|15|240|240|16|GetValue
|Report:valuable|2|13|13|6|GetValue
|Report:ts1|2|9|9|4|GetValue
|Report:ts2|2|7|7|3|GetValue